In [1]:
import numpy as np
import keras 
from keras.models import Sequential, Model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
from keras.applications.densenet import DenseNet121
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
import itertools
import matplotlib.pyplot as plt

Using TensorFlow backend.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([(

In [2]:
train_path = '/content/RASH_INSPECTOR/Dataset/split_eachpart/arm/train'
val_path = '/content/RASH_INSPECTOR/Dataset/split_eachpart/arm/val'
test_path = '/content/RASH_INSPECTOR/Dataset/split_eachpart/arm/test'

In [3]:
image_size = 224
epochs = 50
batch_size = 10

In [4]:
train_generator = ImageDataGenerator(
    preprocessing_function= \
    keras.applications.densenet.preprocess_input).flow_from_directory(
    train_path,
    target_size=(image_size, image_size),
    batch_size=batch_size)

val_generator = ImageDataGenerator(
    preprocessing_function= \
    keras.applications.densenet.preprocess_input).flow_from_directory(
    val_path,
    target_size=(image_size, image_size),
    batch_size=batch_size)

test_generator = ImageDataGenerator(
    preprocessing_function= \
    keras.applications.densenet.preprocess_input).flow_from_directory(
    test_path,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    shuffle=False)

FileNotFoundError: [Errno 2] No such file or directory: '/content/RASH_INSPECTOR/Dataset/split_eachpart/arm/train'